In [17]:
import pickle
import random
import os

In [18]:

base_dir = "../ass4"
models = {
    "unigram": pickle.load(open("unigram_model.pkl", "rb")),
    "bigram": pickle.load(open(os.path.join(base_dir, "bigram_model.pkl"), "rb")),
    "trigram": pickle.load(open(os.path.join(base_dir, "trigram_model.pkl"), "rb")),
    "quadgram": pickle.load(open(os.path.join(base_dir, "quadgram_model.pkl"), "rb")),
}

In [19]:
def tok2str(tok):
    """Convert tuple tokens or plain strings to readable strings"""
    if isinstance(tok, tuple):
        return " ".join(str(t) for t in tok)
    return str(tok)

In [20]:
def get_random_context(model):
    """Pick a random valid context from the model"""
    return random.choice(list(model.keys())) if model else ()

In [21]:
def greedy_generate(model, n, max_len=20, end_token="</s>"):
    if n == 1:  # unigram
        sentence = []
        for _ in range(max_len):
            if not model: break
            next_word = max(model, key=model.get)
            if tok2str(next_word) == end_token:
                break
            sentence.append(tok2str(next_word))
        return " ".join(sentence) if sentence else "<EMPTY>"

    # start with a random context
    context = get_random_context(model)
    sentence = list(context)

    for _ in range(max_len):
        candidates = model.get(context, {})
        candidates = {w: p for w, p in candidates.items() if isinstance(p, (float, int))}
        if not candidates:
            break
        next_word = max(candidates, key=candidates.get)
        if tok2str(next_word) == end_token:
            break
        sentence.append(next_word)
        context = tuple(sentence[-(n - 1):])

    output = [tok2str(tok) for tok in sentence[n - 1:]]
    return " ".join(output) if output else "<EMPTY>"

In [22]:
def beam_search_generate(model, n, beam_size=20, max_len=20, end_token="</s>"):
    if n == 1:  # unigram
        beams = [([], 1.0)]
        for _ in range(max_len):
            new_beams = []
            for seq, score in beams:
                for word, prob in model.items():
                    if not isinstance(prob, (float, int)):
                        continue
                    new_beams.append((seq + [tok2str(word)], score * prob))
            if not new_beams:
                return " ".join(beams[0][0]) if beams else "<EMPTY>"
            beams = sorted(new_beams, key=lambda x: x[1], reverse=True)[:beam_size]
        return " ".join(beams[0][0]) if beams else "<EMPTY>"

    # start with random context
    context = get_random_context(model)
    beams = [(list(context), 1.0)]

    for _ in range(max_len):
        new_beams = []
        for seq, score in beams:
            context = tuple(seq[-(n - 1):])
            candidates = model.get(context, {})
            for word, prob in candidates.items():
                if not isinstance(prob, (float, int)):
                    continue
                new_seq = seq + [word]
                new_score = score * prob
                new_beams.append((new_seq, new_score))
        if not new_beams:
            break
        beams = sorted(new_beams, key=lambda x: x[1], reverse=True)[:beam_size]
        if any(tok2str(seq[-1]) == end_token for seq, _ in beams):
            break

    if not beams:
        return "<EMPTY>"

    best_seq, _ = beams[0]
    output = [tok2str(tok) for tok in best_seq[n - 1:]]
    return " ".join(output) if output else "<EMPTY>"

In [27]:
# Map n-gram names to their 'n'
n_dict = {
    "unigram": 1,
    "bigram": 2,
    "trigram": 3,
    "quadgram": 4
}

for name, model in models.items():
    n = n_dict.get(name)
    if n is None:
        print(f"⚠️ Could not determine n for model '{name}', skipping")
        continue

    print(f"\n=== {name.upper()} MODEL (n={n}) ===")

    print("\nGreedy Samples:")
    for i in range(5):  # set to 100 for full run
        try:
            print(f"{i+1}: {greedy_generate(model, n)}")
        except Exception as e:
            print(f"⚠️ Error generating greedy sample: {e}")

    print("\nBeam Search Samples:")
    for i in range(5):  # set to 100 for full run
        try:
            print(f"{i+1}: {beam_search_generate(model, n)}")
        except Exception as e:
            print(f"⚠️ Error generating beam search sample: {e}")



=== UNIGRAM MODEL (n=1) ===

Greedy Samples:
1:                    
2:                    
3:                    
4:                    
5:                    

Beam Search Samples:
1: 
2: 
3: 
4: 
5: 

=== BIGRAM MODEL (n=2) ===

Greedy Samples:
1: શકે છે અને આ કિસ્સામાં વ્યક્તિગત ઉષ્ણતા બિંદુ ( ) ની જમીન મુક્ત છે અને આ કિસ્સામાં વ્યક્તિગત ઉષ્ણતા બિંદુ
2: સમયના વહેવા સાથે જ નહીં પરંતુ આ કિસ્સામાં વ્યક્તિગત ઉષ્ણતા બિંદુ ( ) ની જમીન મુક્ત છે અને આ કિસ્સામાં
3: મળી હતી અને આ કિસ્સામાં વ્યક્તિગત ઉષ્ણતા બિંદુ ( ) ની જમીન મુક્ત છે અને આ કિસ્સામાં વ્યક્તિગત ઉષ્ણતા બિંદુ
4: ટાળી દેવી યોગ્ય નથી અને આ કિસ્સામાં વ્યક્તિગત ઉષ્ણતા બિંદુ ( ) ની જમીન મુક્ત છે અને આ કિસ્સામાં વ્યક્તિગત
5: ઘરેલુ ઉપયોગમાં લેવાય છે અને આ કિસ્સામાં વ્યક્તિગત ઉષ્ણતા બિંદુ ( ) ની જમીન મુક્ત છે અને આ કિસ્સામાં વ્યક્તિગત

Beam Search Samples:


1: માત્ર ઉજજડ જમીનમાં ૪૦ સોલાર સ્ટ્રીટ લાઈટ હાઉસ પ્રોજેક્ટનું જાન્યુઆરીએ ઈ ખાતમુહૂર્ત પ્રધાનમંત્રી નરેન્દ્રભાઈ મોદીના વરદ હસ્તે ટીડીઓ હિરેન ચૌહાણ
2: ભારે ચકચાર મચી બીબીબી સંવાદદાતા નવી સેન્ટ્રોમા રિયલ વર્લ્ડમાં રજિસ્ટર્ડ ઇન્વેસ્ટમેંટ એડવાઇઝરના કાર્ય અમારી સરકારના મંત્રીઓ પદાધિકારીઓ અધિકારીઓ જેમને સ્થાયી
3: નથી કોંગ્રેસના કાર્યકર્તાઓએ ઝડપ્યો દારૂનો જથ્થો ઝડપાયો તુલા રાશિના જાતકો ભગવાન આપણી વચ્ચેથી અણધારી વિદાય લીધી નાણામંત્રી નિર્માલ સીતારમણના શુક્રવારે
4: આવેલ કોઠી ફળીયુ ન્યુ આશાપુરી નવાયાર્ડ જાદવપાર્ક ન્યુ આશાપુરી નવાયાર્ડ જાદવપાર્ક ન્યુ આશાપુરી નવાયાર્ડ જાદવપાર્ક ન્યુ આશાપુરી નવાયાર્ડ જાદવપાર્ક ન્યુ
5: સેવા ઉદ્યોગો સમીક્ષાઓ ઘણો છોડી ઠેર ઠેર ઠેર ઠેર ઠેર ઠેર ઠેર ઠેર ઠેર ઠેર ઠેર ઠેર રખડવું પડે છે

=== TRIGRAM MODEL (n=3) ===

Greedy Samples:
1: બની જાય છે સંદેશ ન્યૂઝની ગ્રાઉન્ડ ઝીરો રિપોર્ટ સુરતના આ વિસ્તારમાં સૌથી વધુ લોકપ્રિય કીફિર પર અનલોડ છે આ ઉપરાંત
2: દુનિયામાં પ્રખ્યાત છે વીરપુરની ખ્યાતિનું કારણ છે તેમાં આવેલું સુપ્રસિદ્ધ જલારામ મંદિર દર વર્ષે લાખો ભક્તો અહીં જલારામ બાપાના દર્શન


In [ ]:
for name, model in models.items():
    print(f"Processing model: {name}")
    print(f"Model size: {len(model)}")

    n = {"unigram": 1, "bigram": 2}.get(name, 1)
    ...
   #hence problem with unigram model (left to solve yet) 


Processing model: unigram
Model size: 1
Processing model: bigram
Model size: 8264
Processing model: trigram
Model size: 18821
Processing model: quadgram
Model size: 21237
